In [2]:
filename = 'cp_qq_untrim_standard.h5'

import h5py

f = h5py.File(filename, 'r')
list(f.keys())

['images', 'meta_variables']

In [4]:
f['images']

<HDF5 dataset "images": shape (64557, 65, 65), type "<f4">

In [4]:
list(f['meta_variables'].keys())

['ec_1',
 'ec_2',
 'ec_3',
 'jet_delta_R',
 'jet_eta',
 'jet_mass',
 'jet_phi',
 'jet_pt',
 'pull1',
 'pull2',
 'signal',
 'tau_1',
 'tau_2',
 'tau_21',
 'tau_3',
 'tau_32']

In [9]:
f['meta_variables']['ec_1']

<HDF5 dataset "ec_1": shape (64557,), type "<f8">

In [12]:
import time

import h5py
import numpy as np
import sklearn.utils
import os
#from imblearn.under_sampling import RandomUnderSampler

max_chunks = 10000

In [13]:
# Zero pads images from 25x25 to 33x33x1.
def zero_pad(array):
    assert array.shape == (len(array), 25, 25)
    array = np.pad(array, pad_width=([0, 0], [4, 4], [4, 4]),
                   mode='constant', constant_values=0)
    array = array[..., np.newaxis]
    return array

In [19]:
# Implements also large data set method, which splits raw samples into max_size ones, and saves them continuously.
def raw_data_to_ready_data(max_size, gen):
    ts = time.time()

    #fname0, fname1 = get_raw_names()[gen]
    fname0 = "cp_qq_untrim_standard.h5"
    fname1 = "zp_qq_untrim_standard.h5"
    
    #dir_name = drive_path + "ready data/" + gen.replace(' ', '/')
    dir_name = "ready_data"
    if not os.path.exists(dir_name + '/'):
        os.makedirs(dir_name + '/')

    size_0 = len(h5py.File(fname0, 'r')['images'])
    size_1 = len(h5py.File(fname1, 'r')['images'])
    splits = int(np.ceil(max(size_0, size_1) / max_size))  # Celling function

    increment_0 = int(size_0 / splits)
    increment_1 = int(size_1 / splits)

    print(gen)
    print()
    print("Splitting data into %d parts." % splits)
    print("Resulting data sets will be approximately of size %d images." % (min(increment_1, increment_0) * splits))
    print("Training, Validation, Tests combined.")
    print("Increment of 0's taken is %d" % increment_0)
    print("Increment of 1's taken is %d" % increment_1)
    print("--------------------------------------")
    print("Split 1 started")

    ti = time.time()
    first_helper(h5py.File(fname0)['images'][:increment_0],
                 h5py.File(fname1)['images'][:increment_1],
                 dir_name)
    print("Split 1 ended. It took %0.2f seconds." % (time.time() - ti))
    for i in range(1, splits):
        print("Split", i + 1, "started")
        ti = time.time()
        helper(h5py.File(fname0)['images'][i * increment_0:(i + 1) * increment_0],
               h5py.File(fname1)['images'][i * increment_1:(i + 1) * increment_1],
               dir_name)
        print("Split %d ended. It took %0.2f seconds." % i + 1, (time.time() - ti))

    # Print how long it took
    hs = int((time.time() - ts) / 3600)
    ms = int((time.time() - ts) % 3600 / 60)
    s = (time.time() - ts) % 3600 % 60
    print("Time method took was %d hours, %d minutes, %0.2f seconds for %s" % hs, ms, s, dir_name)
    print("")
    test_dimensions(dir_name + "/data.h5")
    
def test_dimensions(fname):
    with h5py.File(fname, 'r') as f:
        for k in f.keys():
            for v in f[k].keys():
                print("%s %s shape is, %s" % k, v, f[k][v].shape)
    
def first_helper(x0, x1, dir_name):
    x0, x1, x0tr, x1tr, x0val, x1val = pre_process(x0, x1)
    with h5py.File(dir_name + '/data.h5', 'w') as h:
        t = h.create_group('test')
        x_test = zero_pad(np.concatenate((x0[x0val:], x1[x1val:])))
        t.create_dataset('x', data=x_test, shape=x_test.shape, maxshape=([None] + list(x_test.shape[1:])))
        y_test = np.concatenate((np.zeros(len(x0[x0val:])), np.ones(len(x1[x1val:]))))
        t.create_dataset('y', data=y_test, shape=y_test.shape, maxshape=[None])

        t = h.create_group('val')
        x_val = zero_pad(np.concatenate((x0[x0tr:x0val], x1[x1tr:x1val])))
        t.create_dataset('x', data=x_val, shape=x_val.shape, maxshape=([None] + list(x_test.shape[1:])))
        y_val = np.concatenate((np.zeros(x0val - x0tr), np.ones(x1val - x1tr)))
        t.create_dataset('y', data=y_val, shape=y_val.shape, maxshape=[None])

    x0 = x0[:x0tr]
    x1 = x1[:x1tr]
    x0 = np.reshape(x0, [len(x0), 625])
    x1 = np.reshape(x1, [len(x1), 625])

    r = RandomUnderSampler()
    x, y = r.fit_sample(np.concatenate((x0, x1)), np.concatenate((np.zeros(len(x0)), np.ones(len(x1)))))
    x, y = sklearn.utils.shuffle(x, y)

    x = zero_pad(np.reshape(x, [len(x), 25, 25]))

    with h5py.File(dir_name + '/data.h5', 'a') as h:
        t = h.create_group('train')
        t.create_dataset('x', data=x, shape=x.shape, maxshape=([None] + list(x_test.shape[1:])))
        t.create_dataset('y', data=y, shape=[len(y)], maxshape=[None])
        
def helper(x0, x1, dir_name):
    x0, x1, x0tr, x1tr, x0val, x1val = pre_process(x0, x1)

    with h5py.File(dir_name + '/data.h5', 'a') as h:
        x_test = zero_pad(np.concatenate((x0[x0val:], x1[x1val:])))
        h["test/x"].resize((h["test/x"].shape[0] + len(x_test)), axis=0)
        h["test/x"][-len(x_test):] = x_test

        y_test = np.concatenate((np.zeros(len(x0[x0val:])), np.ones(len(x1[x1val:]))))
        h['test/y'].resize((h["test/y"].shape[0] + len(y_test)), axis=0)
        h["test/y"][-len(y_test):] = y_test

        x_val = zero_pad(np.concatenate((x0[x0tr:x0val], x1[x1tr:x1val])))
        h["val/x"].resize((h["val/x"].shape[0] + len(x_val)), axis=0)
        h["val/x"][-len(x_val):] = x_val

        y_val = np.concatenate((np.zeros(x0val - x0tr), np.ones(x1val - x1tr)))
        h['val/y'].resize((h["val/y"].shape[0] + len(y_val)), axis=0)
        h["val/y"][-len(y_val):] = y_val

    x0 = x0[:x0tr]
    x1 = x1[:x1tr]
    x0 = np.reshape(x0, [len(x0), 625])
    x1 = np.reshape(x1, [len(x1), 625])

    r = RandomUnderSampler()
    x, y = r.fit_sample(np.concatenate((x0, x1)), np.concatenate((np.zeros(len(x0)), np.ones(len(x1)))))
    x, y = sklearn.utils.shuffle(x, y)

    x = zero_pad(np.reshape(x, [len(x), 25, 25]))
    with h5py.File(dir_name + '/data.h5', 'a') as h:
        h["train/x"].resize((h["train/x"].shape[0] + len(x)), axis=0)
        h["train/x"][-len(x):] = x

        h['train/y'].resize((h["train/y"].shape[0] + len(y)), axis=0)
        h["train/y"][-len(y):] = y
        
def pre_process(x0, x1):
    x0 = np.reshape(x0, [len(x0), 625])
    x1 = np.reshape(x1, [len(x1), 625])

    x0 = sklearn.utils.shuffle(x0)
    x1 = sklearn.utils.shuffle(x1)

    x0 = np.reshape(x0, [len(x0), 25, 25])
    x1 = np.reshape(x1, [len(x1), 25, 25])

    x0tr = int(len(x0) * 0.6)
    x1tr = int(len(x1) * 0.6)
    x0val = int(len(x0) * 0.8)
    x1val = int(len(x1) * 0.8)
    return x0, x1, x0tr, x1tr, x0val, x1val

In [20]:
raw_data_to_ready_data(max_chunks, 'Herwig Dipole')

Herwig Dipole

Splitting data into 17 parts.
Resulting data sets will be approximately of size 64549 images.
Training, Validation, Tests combined.
Increment of 0's taken is 3797
Increment of 1's taken is 9701
--------------------------------------
Split 1 started


ValueError: cannot reshape array of size 16042325 into shape (3797,625)